In [2]:
def create_blackboard_file():
    from IPython.display import display, HTML, clear_output
    import ipywidgets as widgets
    import pandas as pd
    import current_course as cc
    import import_ipynb
    import GradingTools as gt


    #setup
    course = cc.read_current_course()
    if course is None:
        raise Exception("You must select a current course first")

    templates = gt.get_templates()
    assignments = gt.client.get_course_assignments(course)
    roster = gt.client.get_course_roster(course)
    units = assignments['unit_name'].unique().tolist()
    assignment_names = assignments['assignment_name'].unique().tolist()
    gt.initialize_directories()

    # widgets
    current_course_text = widgets.Text(
        value=course,
        description='Course:',
        disabled=True
    )

    unit_dropdown = widgets.Dropdown(
        options=units,
        value=units[0],
        description='Lesson:',
        layout={'width': 'max-content'},
        disabled=False
    )

    assignment_name_dropdown = widgets.Dropdown(
        options=assignment_names,
        value=assignment_names[0],
        description='Assignment:',
        layout={'width': 'max-content'},
        disabled=False
    )

    assignment_date_text = widgets.Text(
        value=assignments[ assignments['assignment_name'] == assignment_name_dropdown.value]['due_date'].values[0],
        description='Due Date:',
        disabled=True
    )

    assignment_points_text = widgets.Text(
        value=str(assignments[ assignments['assignment_name'] == assignment_name_dropdown.value]['points'].values[0]),
        description='Total Points:',
        layout={'width': 'max-content'},
        disabled=True
    )

    assignment_lms_column_text = widgets.Text(
        value=str(assignments[ assignments['assignment_name'] == assignment_name_dropdown.value]['lms_column_id'].values[0]),
        description='BB Column:',
        layout={'width': 'max-content'},
        disabled=True
    )

    create_file_button = widgets.Button(
        description="Create Blackboard File",
        icon="save",
        button_style='success',
        layout={'width': 'max-content'},
        disabled = False)

    output = widgets.Output()

    # functions
    def unit_dropdown_on_change(*args):
        assignment_name_dropdown.options = assignments[assignments['unit_name'] == unit_dropdown.value].sort_values('assignment_name')['assignment_name'].unique().tolist()

    def assignment_name_dropdown_on_change(*args):
        assignment_date_text.value = assignments[ assignments['assignment_name'] == assignment_name_dropdown.value]['due_date'].values[0]
        assignment_points_text.value=str(assignments[ assignments['assignment_name'] == assignment_name_dropdown.value]['points'].values[0])
        assignment_lms_column_text.value=str(assignments[ assignments['assignment_name'] == assignment_name_dropdown.value]['lms_column_id'].values[0])

    def create_file_button_on_click(*args):
        count, bbfilename, df = gt.create_bb_file(course, assignment_name_dropdown.value)
        with output:
            clear_output()
            display(HTML(f"<p>{count} grades written.</p>"))
            display(HTML(f"<p>File: {bbfilename}</p>"))
            display(df)


    # handlers
    unit_dropdown.observe(unit_dropdown_on_change)
    assignment_name_dropdown.observe(assignment_name_dropdown_on_change)
    create_file_button.on_click(create_file_button_on_click)


    # layouts
    display(HTML(f"<h3>Create Blackboard File</h3>"))
    display(current_course_text)
    display(widgets.HBox([unit_dropdown,assignment_name_dropdown]))
    display(assignment_date_text)
    display(assignment_points_text)
    display(assignment_lms_column_text)
    display(widgets.HBox([create_file_button]))
    display(output)


    #initialize
    unit_dropdown_on_change()
    assignment_name_dropdown_on_change()



In [3]:
if __name__ == '__main__':
    create_blackboard_file()

Text(value='tst101-spring2024', description='Course:', disabled=True)

Text(value='2024-01-05 23:59', description='Due Date:', disabled=True)

Text(value='5', description='Total Points:', disabled=True, layout=Layout(width='max-content'))

Text(value='342634', description='BB Column:', disabled=True, layout=Layout(width='max-content'))

Output()